In [5]:
import torch
import PIL
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torchvision
from torch.utils.data import TensorDataset, DataLoader

AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [4]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")


NameError: name 'torchvision' is not defined

In [2]:
# pool of square window of size=3, stride=2
m = torch.nn.AvgPool2d(3, stride=2)
# pool of non-square window
m = torch.nn.AvgPool2d((3, 2), stride=(2, 1))
input = torch.randn(20, 16, 50, 32)
output = m(input)
output.shape


torch.Size([20, 16, 24, 31])

In [3]:
model = torch.hub.load("pytorch/vision", "resnet50", weights="IMAGENET1K_V2")

Using cache found in C:\Users\Mike/.cache\torch\hub\pytorch_vision_main


ValueError: Could not find the operator torchvision::nms. Please make sure you have already registered the operator and (if registered from C++) loaded it via torch.ops.load_library.

In [ ]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
for p in model.parameters():
    p.requires_grad = False

In [ ]:
class Sim(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__(Sim, self).__init__()

    def forward(self, x):
        return x

In [56]:
# model.avgpool = torch.nn.Conv3d()

In [62]:
m = torch.nn.Conv1d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)
output.shape

torch.Size([20, 33, 24])

In [33]:
model.fc = torch.nn.Sequential(
    torch.nn.Linear(2048, 1024),
    torch.nn.Tanh(),
    torch.nn.Linear(1024, 256),
    torch.nn.Tanh(),
    torch.nn.Linear(256, 64),
    torch.nn.Tanh(),
    torch.nn.Linear(64, 4),
    torch.nn.Tanh(),
    torch.nn.Linear(4, 1),
    torch.nn.ReLU()
)

In [34]:
model.to('cuda')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [38]:
data = np.load('datasets/images.npy/images.npy')

In [40]:
labels = pd.read_csv('datasets/labels.csv')['count']

In [43]:
data.shape

(2000, 480, 640, 3)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(data, labels.to_numpy(), test_size=0.15,)

In [64]:

# import dill
# with open('./env_of_customer/Env.pkl', 'rb') as f:
#     Env = dill.load(f)
# with open('./env_of_customer/get_propability_of_walk_age.pkl', 'rb') as f:
#     get_propability_of_walk_age = dill.load(f)
# with open('./env_of_customer/get_propability_for_checkout_transport.pkl', 'rb') as f:
#     get_propability_for_checkout_transport = dill.load(f)
# with open('./env_of_customer/change_distance_by_transport.pkl', 'rb') as f:
#     change_distance_by_transport = dill.load(f)
# with open('models/raiting_check/raiting_checkout_pred_tree.pkl', 'rb') as f:
#     raiting_checkout_pred_tree = dill.load(f)
# with open('models/raiting_check/pf.pkl', 'rb') as f:
#     pf = dill.load(f)
# with open('models/raiting_check/dataset_preprocessor_for_raiting.pkl', 'rb') as f:
#     dataset_preprocessor_for_raiting = dill.load(f)
# with open('models/raiting_check/std_scaler_for_raiting.pkl', 'rb') as f:
#     std_scaler_for_raiting = dill.load(f)
# with open('./env_of_customer/from_vector_to_dict.pkl', 'rb') as f:
#     from_vector_to_dict = dill.load(f)
# with open('./NNmodel/model_pred.pkl', 'rb') as f:
#     model_pred = dill.load(f)
# with open('./NNmodel/model.pkl', 'rb') as f:
#     model= dill.load(f)
    
# env = Env(get_propability_for_checkout_transport=get_propability_for_checkout_transport,
# change_distance_by_transport=change_distance_by_transport,
# get_propability_of_walk_age=get_propability_of_walk_age,
# propability_of_checkout_branch_by_raiting=lambda a: raiting_checkout_pred_tree.predict(dataset_preprocessor_for_raiting([a], pf, std_scaler_for_raiting))*100)
# # loss = torch.nn.MultiLabelSoftMarginLoss()
# # optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
# # model_pred(
# #     model=model,
# #     loss=loss,
# #     data=objects,
# #     from_vector_to_dict=from_vector_to_dict,
# #     env=env,
# # )

In [67]:
from torchvision import transforms
transformsf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),]
)

In [49]:
train_ds = TensorDataset(
    torch.from_numpy(X_train).type(torch.float32).to('cuda'), 
    torch.from_numpy(y_train).type(torch.float32).to('cuda')
)
train_dl = DataLoader(train_ds, batch_size=512, shuffle=True)

In [50]:
test_ds = TensorDataset(torch.from_numpy(X_test).to('cuda'), torch.from_numpy(y_test).to('cuda'))
test_dl = DataLoader(test_ds, batch_size=512, shuffle=True)

In [52]:
loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [80]:
import tensorflow as tf

In [81]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping: no known devices.


In [83]:
epochs = 50
for epoch in range(epochs):
    for x_b, y_b in train_dl:
        outputs = model(transformsf(x_b.to('cuda'))).to('cuda')
        loss_value = loss(outputs, y_b.to('cuda'))
        loss_value.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')

RuntimeError: [enforce fail at alloc_cpu.cpp:80] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1887436800 bytes.